## Task a) Scrape the Family Tree of British Monarchs
1. Scrape the family tree of British monarchs from the Wikipedia page using the **BeautifulSoup** library.
2. Store the scraped monarchs' data in an SQLite table `british_monarch_family_tree` with the following fields:
- `id`: Primary key
- `name`: Monarch's name
- `wiki_url`: Monarch’s Wikipedia URL

In [ ]:
import requests
from bs4 import BeautifulSoup
user_agent = {'User-agent': 'Mozilla/5.0'}

# Retrieve the page below
url = 'https://en.wikipedia.org/w/index.php?title=Family_tree_of_British_monarchs&oldid=1043575587'
page = requests.get(url, headers = user_agent)
soup = BeautifulSoup(page.content)

# Check
#soup 

In [ ]:
table = soup.find('table')
tbody = table.find('tbody')
tagA = tbody.find_all('a')

In [ ]:
monarchs = [] 
for a in tagA:
    if a.find('img'):
        pass
    elif ("Duke of Windsor" in a.text) or ("House" in a.text) or ("House" in a.attrs['href']):
        pass
    else:
        monarchs.append({'name':a.attrs['title'], 'wiki_url':a.attrs['href']})
        
len(monarchs) # Checks monarch - a list of dict

In [ ]:
# Handling George III redirecting of URL

for monarch in monarchs:
    if monarch['name'] == 'George III of the United Kingdom' or monarch['name'] == 'George III':
        monarch['name'] = 'George III'
        monarch['wiki_url'] = '/wiki/George_III'  # Replace with the desired URL
        break  # Exit the loop once the update is made

In [ ]:
import sqlite3
conn = sqlite3.connect('British_Monarchs.db')

cur = conn.cursor()
cur.execute('DROP TABLE IF EXISTS british_monarch_family_tree ')
cur.execute('CREATE TABLE british_monarch_family_tree (id INTEGER PRIMARY KEY AUTOINCREMENT, name TEXT, wiki_url TEXT)')

# commit SQL statement to database
conn.commit()

In [ ]:
for item in monarchs:
    cur.execute('INSERT INTO british_monarch_family_tree (name, wiki_url) VALUES (?, ?)',(item['name'], item['wiki_url']))
conn.commit()

## Task b) Add Parent Information to the Family Tree
1. Modify the `british_monarch_family_tree` table to include two new fields:
- `father_id`: Foreign key referencing the `id` of the monarch’s father
- `mother_id`: Foreign key referencing the `id` of the monarch’s mother
2. Scrape the father and mother information from the respective monarch's Wikipedia pages.
3. If no information is available, set `null` values for missing parents.
4. If a parent does not exist in the table, add them and set the relationship (`father_id` and `mother_id`).

In [ ]:
# Adding 2 new field: father_id and mother_id
cur.execute('ALTER TABLE british_monarch_family_tree ADD father_id INTEGER')
cur.execute('ALTER TABLE british_monarch_family_tree ADD mother_id INTEGER')

Namelist = {}

cur.execute('SELECT wiki_url,id FROM british_monarch_family_tree')
myresult = cur.fetchall()
for row in myresult:
    Namelist[row[0]]=row[1]

In [ ]:
key_iterable = Namelist.keys()
key_list = list(key_iterable)

In [ ]:
# Updating Record with Father's and Mother's ID

for i in key_list:
    domain = 'https://en.wikipedia.org'
    page = requests.get(domain + i, headers=user_agent)
    
    # Check if the request was successful
    if page.status_code != 200:
        print(f"Failed to retrieve {i}")
        continue  # Skip to the next iteration if the page is unavailable
    
    soup = BeautifulSoup(page.content, 'html.parser')

    # Initialize variables to avoid potential issues
    FatherName = None
    FatherURL = None
    MotherName = None
    MotherURL = None

    # Scraping for Father and Mother
    table = soup.find('table', {'class': 'infobox vcard'})
    if table:
        tbody = table.find('tbody')
        tr = tbody.find_all('tr') if tbody else []

        for r in tr:
            H = r.find_all('th')
            for h in H:
                if h.text == 'Father':
                    TD = r.find_all('td')
                    for td in TD:
                        if td.find('a'):
                            FatherName = td.find('a')['title']
                            FatherURL = td.find('a')['href']
                elif h.text == 'Mother':
                    TD = r.find_all('td')
                    for td in TD:
                        if td.find('a'):
                            MotherName = td.find('a')['title']
                            MotherURL = td.find('a')['href']

    # Update or Insert Father if necessary
    if FatherURL:
        if FatherURL in Namelist:
            FatherID = Namelist[FatherURL]
        else:
            cur.execute('INSERT INTO british_monarch_family_tree (name, wiki_url) VALUES (?, ?)', (FatherName, FatherURL))
            cur.execute('SELECT id FROM british_monarch_family_tree WHERE wiki_url = ?', (FatherURL,))
            FatherID = cur.fetchone()[0]
            Namelist[FatherURL] = FatherID

    # Update or Insert Mother if necessary
    if MotherURL:
        if MotherURL in Namelist:
            MotherID = Namelist[MotherURL]
        else:
            cur.execute('INSERT INTO british_monarch_family_tree (name, wiki_url) VALUES (?, ?)', (MotherName, MotherURL))
            cur.execute('SELECT id FROM british_monarch_family_tree WHERE wiki_url = ?', (MotherURL,))
            MotherID = cur.fetchone()[0]
            Namelist[MotherURL] = MotherID

    # Ensure FatherID and MotherID are not None and update the database
    if FatherID is not None and MotherID is not None:
        # Explicitly convert FatherID and MotherID to integers
        Updatequery = 'UPDATE british_monarch_family_tree SET father_id = ?, mother_id = ? WHERE wiki_url = ?'
        cur.execute(Updatequery, (int(FatherID), int(MotherID), i))

conn.commit()


## Task c) Query Children of King "George III"
- Use SQLite3 to find and return all children of King George III using his Wikipedia page [here](https://en.wikipedia.org/wiki/George_III).

In [ ]:
url = '/wiki/George_III'

# Find George III id
query = 'SELECT id, wiki_url FROM british_monarch_family_tree WHERE wiki_url = "{}"'.format(url)
cur.execute(query)
myresult = cur.fetchall()
for row in myresult:
    GeorgeIII_id = row[0]

# Find all rows with father_id as George III's id
GeorgeIII_Children = []
query = 'SELECT name, father_id, wiki_url FROM british_monarch_family_tree WHERE father_id = "{}"'.format(GeorgeIII_id)
cur.execute(query)
myresult = cur.fetchall()


if len(myresult) == 0:
    print('None Found')
else:
    print("The following are children of King George III:")
    for row in myresult:
        print("-",row[0])        


## Task d) Query Father and Mother of King "George III"
- Use SQLite3 to find the father and mother of King George III from the database.

In [ ]:
# Apply SQLite3 library to find the father and mother of King "George III"

# Obtaining George III father's and mother's id
Parentquery = 'SELECT name, father_id, mother_id FROM british_monarch_family_tree WHERE id = "{}"'.format(GeorgeIII_id)
cur.execute(Parentquery)
myresult = cur.fetchall()
for row in myresult:
    GeorgeIII_fatherId = row[1]
    GeorgeIII_motherId = row[2]

# Finding George's Parents with the respective obtained IDs
GeorgeIII_Parents = []
Fatherquery = 'SELECT name FROM british_monarch_family_tree WHERE id = "{}"'.format(GeorgeIII_fatherId)
cur.execute(Fatherquery)
myresult = cur.fetchall()
for row in myresult:
    GeorgeIII_Parents.append(row[0])
Motherquery = 'SELECT name FROM british_monarch_family_tree WHERE id = "{}"'.format(GeorgeIII_motherId)
cur.execute(Motherquery)
myresult = cur.fetchall()
for row in myresult:
    GeorgeIII_Parents.append(row[0])

# Answer
print("The following are parents of King George III: \n Father: {}\n Mother: {}".format(GeorgeIII_Parents[0],GeorgeIII_Parents[1]))

## Task e) Query Siblings of King "George IV"
- Use SQLite3 to find all siblings of King George IV using his Wikipedia page [here](https://en.wikipedia.org/wiki/George_IV).

In [ ]:
# IMPT! -> On the Wikipedia Page, '/wiki/George_IV' has been updated to be redirected from '/wiki/George_IV_of_the_United_Kingdom'
GeorgeIVurl = '/wiki/George_IV_of_the_United_Kingdom'

# Find George IV's parents' IDs
Parentquery = 'SELECT name, father_id, mother_id FROM british_monarch_family_tree WHERE wiki_url = "{}"'.format(GeorgeIVurl)
cur.execute(Parentquery)
myresult = cur.fetchall()
for row in myresult:
    GeorgeIV_fatherId = row[1]
    GeorgeIV_motherId = row[2]

GeorgeIV_Siblings = []
query = 'SELECT name FROM british_monarch_family_tree WHERE wiki_url != "{}" AND (father_id = "{}" OR mother_id = "{}")'.format(GeorgeIVurl, GeorgeIV_fatherId, GeorgeIV_motherId)
cur.execute(query)
myresult = cur.fetchall()
for row in myresult:
    GeorgeIV_Siblings.append(row[0])

# Answer
print("The following are siblings of King George IV:")
for sibling in GeorgeIV_Siblings:
    print("-",sibling)

## Task f) Query Descendants of "Queen Victoria"
- Use SQLite3 and Pandas DataFrame to find all descendants of Queen Victoria from the database.

In [ ]:
import pandas as pd

df = pd.read_sql_query("SELECT * from british_monarch_family_tree", conn)

# Find Queen Victoria Row Index
df.loc[df['name'] == 'Queen Victoria']

In [ ]:
AfterQueenVictoria = df.loc[87+1:]
QueenVictoriaDescendantsDF = AfterQueenVictoria.dropna()
QueenVictoriaDescendantsDF.reset_index(drop=True, inplace=True)
QueenVictoriaDescendantsDF